# データサイエンス100本ノック（構造化データ加工編） - Python

### 独自にライブラリを追加

In [ ]:
!pip install MonthDelta

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from imblearn.under_sampling import RandomUnderSampler
from monthdelta import monthmod
import re

if 'PG_PORT' in os.environ:
    pgconfig = {
        'host': 'db',
        'port': os.environ['PG_PORT'],
        'database': os.environ['PG_DATABASE'],
        'user': os.environ['PG_USER'],
        'password': os.environ['PG_PASSWORD'],
    }


    # pd.read_sql用のコネクタ
    conn = psycopg2.connect(**pgconfig)

    df_customer = pd.read_sql(sql='select * from customer', con=conn)
    df_category = pd.read_sql(sql='select * from category', con=conn)
    df_product = pd.read_sql(sql='select * from product', con=conn)
    df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
    df_store = pd.read_sql(sql='select * from store', con=conn)
    df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

else:
    if not os.path.exists('data/'):
        !git clone https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess
        os.chdir('100knocks-preprocess/docker/work/')

    dtype = {
        'customer_id': str,
        'gender_cd': str,
        'postal_cd': str,
        'application_store_cd': str,
        'status_cd': str,
        'category_major_cd': str,
        'category_medium_cd': str,
        'category_small_cd': str,
        'product_cd': str,
        'store_cd': str,
        'prefecture_cd': str,
        'tel_no': str,
        'postal_cd': str,
        'street': str
    }

    df_customer = pd.read_csv("data/customer.csv", dtype=dtype)
    df_category = pd.read_csv("data/category.csv", dtype=dtype)
    df_product = pd.read_csv("data/product.csv", dtype=dtype)
    df_receipt = pd.read_csv("data/receipt.csv", dtype=dtype)
    df_store = pd.read_csv("data/store.csv", dtype=dtype)
    df_geocode = pd.read_csv("data/geocode.csv", dtype=dtype)

# 演習問題

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [ ]:
df_receipt.head(10)

---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [ ]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].head(10)

---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdsales_dateに項目名を変更しながら抽出すること。

In [ ]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].rename(columns = {'sales_ymd': 'sales_date'}).head(10)

---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [ ]:
df_receipt[df_receipt['customer_id'] == 'CS018205000001'][['sales_ymd', 'customer_id', 'product_cd', 'amount']]

---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [ ]:
df_receipt[(df_receipt['customer_id'] == 'CS018205000001') & (df_receipt['amount'] >= 1000)][['sales_ymd', 'customer_id', 'product_cd', 'amount']]

---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
df_receipt[(df_receipt['customer_id'] == 'CS018205000001') & ((df_receipt['amount'] >= 1000) | (df_receipt['quantity'] >= 5))][['sales_ymd', 'customer_id', 'product_cd', 'amount']]

---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [ ]:
# df[df[～]]の範囲抽出ではbetweenに該当する機能なし
df_receipt[(df_receipt['customer_id'] == 'CS018205000001') & ((df_receipt['amount'] >= 1000) & (df_receipt['amount'] <= 2000))][['sales_ymd', 'customer_id', 'product_cd', 'amount']]

---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [ ]:
df_receipt[(df_receipt['customer_id'] == 'CS018205000001') & (df_receipt['product_cd'] != 'P071401019')][['sales_ymd', 'customer_id', 'product_cd', 'amount']]

---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
> 
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [ ]:
df_store.query('prefecture_cd != "13" & floor_area <= 900')

---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [ ]:
# strによるトリミングは開始位置と終了位置の指定に注意
# 参考：https://pg-chain.com/python-str-substring#toc3
df_store[df_store['store_cd'].str[:3] == 'S14'].head(10)

---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [ ]:
df_customer[df_customer['customer_id'].str[-1:] == '1'].head(10)

---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [ ]:
df_store[df_store['address'].str.contains('横浜市')]

# 正規表現でも抽出可能
# df_store[df_store['address'].str.match('.*横浜市.*')]

---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [ ]:
df_customer[df_customer['status_cd'].str.match('^[A-F].*')].head(10)

---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [ ]:
df_customer[df_customer['status_cd'].str.match('.*[1-9]$')].head(10)

---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [ ]:
df_customer[df_customer['status_cd'].str.match('^[A-F].*[1-9]$')].head(10)

---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [ ]:
df_store[df_store['tel_no'].str.match('^\d{3}-\d{3}-\d{4}$')]

---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [ ]:
df_customer.sort_values('birth_day').head(10)

---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [ ]:
df_customer.sort_values('birth_day', ascending = False).head(10)

---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [ ]:
df_019 = df_receipt[['customer_id', 'amount']].copy()
df_019['amount_rank'] = df_019['amount'].rank(method = 'min', ascending = False)

df_019.sort_values('amount_rank').head(10)

---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [ ]:
df_020 = df_receipt[['customer_id', 'amount']].copy()
df_020['amount_rank'] = df_020['amount'].rank(method = 'first', ascending = False)

df_020.sort_values('amount_rank').head(10)

---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [ ]:
len(df_receipt)

---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
len(df_receipt['customer_id'].unique())

---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [ ]:
df_receipt[['store_cd', 'amount', 'quantity']].groupby('store_cd').agg(sum_amount = ('amount', 'sum'), sum_quantity = ('quantity', 'sum')).reset_index()

---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt[['customer_id', 'sales_ymd']].groupby('customer_id').agg(max_sales_ymd = ('sales_ymd', 'max')).reset_index().head(10)

---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt[['customer_id', 'sales_ymd']].groupby('customer_id').agg(min_sales_ymd = ('sales_ymd', 'min')).reset_index().head(10)

---
> P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [ ]:
df_026 = df_receipt[['customer_id', 'sales_ymd']].copy()
df_026_grouped = df_026.groupby('customer_id').agg(max_sales_ymd = ('sales_ymd', 'max'), min_sales_ymd = ('sales_ymd', 'min')).reset_index()

df_026_grouped[df_026_grouped['max_sales_ymd'] != df_026_grouped['min_sales_ymd']].head(10)

---
> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [ ]:
df_027 = df_receipt[['store_cd', 'amount']].copy()
df_027_grouped = df_027.groupby('store_cd').agg(mean_amount = ('amount', 'mean')).reset_index()

df_027_grouped.sort_values('mean_amount', ascending = False).head(5)

---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [ ]:
df_028 = df_receipt[['store_cd', 'amount']].copy()
df_028_grouped = df_028.groupby('store_cd').agg(median_amount = ('amount', 'median')).reset_index()

df_028_grouped.sort_values('median_amount', ascending = False).head(5)

---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [ ]:
# GroupByで標準提供されていない統計量を求める場合は、df.groupby('集約キー')['集計対象'].apply(lambda x: 統計量算出処理)
df_029 = df_receipt[['store_cd', 'product_cd']].copy()
df_029_grouped = df_029.groupby('store_cd')['product_cd'].apply(lambda x: x.mode()).reset_index()
df_029_grouped.columns = ['store_cd', 'mode', 'product_cd']

df_029_grouped.head(10)

---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [ ]:
# 標本分散
df_030 = df_receipt[['store_cd', 'amount']].copy()
df_030_grouped = df_030.groupby('store_cd')['amount'].apply(lambda x: x.var(ddof = 0)).reset_index()
df_030_grouped.columns = ['store_cd', 'var_amount']

df_030_grouped.sort_values('var_amount', ascending = False).head(5)

---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

TIPS:

PandasとNumpyでddofのデフォルト値が異なることに注意しましょう
```
Pandas：
DataFrame.std(self, axis=None, skipna=None, level=None, ddof=1, numeric_only=None, **kwargs)
Numpy:
numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=)
```

In [ ]:
# 標本標準偏差
df_031 = df_receipt[['store_cd', 'amount']]
df_031_grouped = df_031.groupby('store_cd')['amount'].apply(lambda x: x.std(ddof = 0)).reset_index()
df_031_grouped.columns = ['store_cd', 'std_amount']

df_031_grouped.sort_values('std_amount', ascending = False).head(5)

---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [ ]:
df_receipt['amount'].quantile([0.25, 0.5, 0.75, 1.0])

---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [ ]:
df_033 = df_receipt[['store_cd', 'amount']].copy()
df_033_grouped = df_033.groupby('store_cd').agg(mean_amount = ('amount', 'mean')).reset_index()

df_033_grouped[df_033_grouped['mean_amount'] >= 330]

---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [ ]:
df_034 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'][['customer_id', 'amount']].copy()
df_034_grouped = df_034.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()
df_034_grouped['sum_amount'].mean()

---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [ ]:
df_035 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'][['customer_id', 'amount']].copy()
df_035_grouped = df_035.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()
mean_sum_amount = df_035_grouped['sum_amount'].mean()

df_035_grouped[df_035_grouped['sum_amount'] >= mean_sum_amount].head(10)

---
> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [ ]:
pd.merge(df_receipt, df_store[['store_cd', 'store_name']], on = 'store_cd', how = 'inner').head(10)

---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [ ]:
pd.merge(df_product, df_category[['category_small_cd', 'category_small_name']], on = 'category_small_cd', how = 'inner').head(10)

---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [ ]:
df_038_c = df_customer[(df_customer['gender_cd'] == '1') & (df_customer['customer_id'].str[:1] != 'Z')][['customer_id']].copy()
df_038_r = df_receipt[['customer_id', 'amount']].copy()

df_038 = pd.merge(df_038_c, df_038_r, on = 'customer_id', how = 'left').fillna(0)
df_038_grouped = df_038.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()
df_038_grouped.head(10)

---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [ ]:
df_039 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'].copy()

df_039_grouped_sum_amount = df_039.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()
df_039_grouped_sum_amount = df_039_grouped_sum_amount.sort_values('sum_amount', ascending = False).head(20)

# 売上日数の多い顧客を算出するために、cutomer_idとsales_ymdで一意にしたうえで集約する
df_039_unique = df_039[['customer_id', 'sales_ymd']].drop_duplicates().copy()
df_039_grouped_count_sales = df_039_unique.groupby('customer_id').agg(count_sales_ymd = ('sales_ymd', 'count')).reset_index()
df_039_grouped_count_sales = df_039_grouped_count_sales.sort_values('count_sales_ymd', ascending = False).head(20)

pd.merge(df_039_grouped_sum_amount, df_039_grouped_count_sales, on = 'customer_id', how = 'outer').sort_values('customer_id')

---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [ ]:
# pandasのmerge（join）にはクロス結合を直接行えない
# 全レコードに共通キーを持たせ、full outer joinする
def cross_join(df_x, df_y):
    
    df_x['tmp'] = 1
    df_y['tmp'] = 1
    
    return pd.merge(df_x, df_y, how = 'outer')

len(cross_join(df_store, df_product))

---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [ ]:
df_receipt[['sales_ymd', 'amount']].groupby('sales_ymd').agg(sum_amount = ('amount', 'sum')) \
    .sort_values('sales_ymd').diff().head(10)

---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [ ]:
df_042 = df_receipt[['sales_ymd', 'amount']]
df_042_grouped = df_042.groupby('sales_ymd').agg(sum_amount = ('amount', 'sum')).sort_values('sales_ymd').reset_index()

for index in range(3, len(df_042_grouped)):
    for lag in range(1,4):
        col = 'lag' + str(lag) + '_sum_amount'
        df_042_grouped.loc[index, [col]] = df_042_grouped.at[index - lag, 'sum_amount']

df_042_grouped.dropna().head(10)

---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [ ]:
df_043_c = df_customer[['customer_id', 'gender', 'age']].copy()
df_043_c['era'] = df_043_c['age'].apply(lambda x: '10代未満' if x < 10 else str(int(x / 10)) + '0代')
df_043_c.drop('age', axis = 1, inplace = True)

df_043_r = df_receipt[['customer_id', 'amount']].copy()

df_043 = pd.merge(df_043_c, df_043_r, on = 'customer_id', how = 'inner')

pd.pivot_table(df_043, index = 'era', columns = 'gender', aggfunc = np.sum)

---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [ ]:
df_044_c = df_customer[['customer_id', 'gender', 'age']].copy()
df_044_c['era'] = df_044_c['age'].apply(lambda x: '10代未満' if x < 10 else str(int(x / 10)) + '0代')
df_044_c.drop('age', axis = 1, inplace = True)

df_044_c['gender'].replace(['東京', '女性', '不明'], ['00', '01', '99'])

df_044_r = df_receipt[['customer_id', 'amount']].copy()

df_044 = pd.merge(df_044_c, df_044_r, on = 'customer_id', how = 'inner')

df_044.groupby(['gender', 'era']).agg(sum_amount = ('amount', 'sum')).reset_index()

---
> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [ ]:
df_045 = df_customer[['customer_id', 'birth_day']].copy()
df_045.loc[:, 'birth_day'] = df_045['birth_day'].astype(str).str.replace('-', '')
df_045.head(10)

---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [ ]:
df_046 = df_customer[['customer_id', 'application_date']].copy()
df_046.loc[:, 'application_date'] = pd.to_datetime(df_046['application_date'])
df_046.head(10)

---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [ ]:
df_047 = df_receipt[['sales_ymd', 'receipt_no', 'receipt_sub_no']].copy()
df_047.loc[:, 'sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'].astype(str))
df_047.head(10)

---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [ ]:
df_048 = df_receipt[['sales_epoch', 'receipt_no', 'receipt_sub_no']].copy()
df_048['convert_sales_epoch'] = df_048['sales_epoch'].apply(datetime.fromtimestamp)
df_048.head(10)

---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [ ]:
df_049 = df_receipt[['sales_epoch', 'receipt_no', 'receipt_sub_no']].copy()
df_049['convert_sales_epoch_y'] = df_049['sales_epoch'].apply(datetime.fromtimestamp).dt.year
df_049[['receipt_no', 'receipt_sub_no', 'convert_sales_epoch_y']].head(10)

---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [ ]:
df_050 = df_receipt[['sales_epoch', 'receipt_no', 'receipt_sub_no']].copy()
df_050['convert_sales_epoch_m'] = df_050['sales_epoch'].apply(datetime.fromtimestamp).dt.month.astype(str).str.zfill(2)
df_050[['convert_sales_epoch_m', 'receipt_no', 'receipt_sub_no']].head(10)

---
> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [ ]:
df_050 = df_receipt[['sales_epoch', 'receipt_no', 'receipt_sub_no']].copy()
df_050['convert_sales_epoch_d'] = df_050['sales_epoch'].apply(datetime.fromtimestamp).dt.day.astype(str).str.zfill(2)
df_050[['convert_sales_epoch_d', 'receipt_no', 'receipt_sub_no']].head(10)

---
> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_052 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'][['customer_id', 'amount']].copy()
df_052_grouped = df_052.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()
df_052_grouped['sum_amount > 2000'] = df_052_grouped['sum_amount'].apply(lambda x: 1 if x > 2000 else 0)
df_052_grouped.head(10)

---
> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [ ]:
df_053_c = df_customer[['customer_id', 'postal_cd']].copy()
df_053_c['tokyo_flag'] = df_053_c['postal_cd'].str[:3].astype(int).apply(lambda x: 1 if 100 <= x <= 209 else 0)

df_053_r = df_receipt['customer_id'].drop_duplicates().copy()

df_053 = pd.merge(df_053_c, df_053_r, on = 'customer_id', how = 'inner')
df_053.groupby('tokyo_flag').agg(count_customer_id = ('customer_id', 'count')).reset_index()

---
> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [ ]:
pattern = '東京都|北海道|(京都|大阪)府|.{2,3}県'
df_054 = df_customer[['address']].copy()
df_054['address_code'] = df_054['address'].apply(lambda x: re.match(pattern, x).group()).replace(['埼玉県', '千葉県', '東京都', '神奈川県'], [11, 12, 13, 14])
df_054.head(10)

---
> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [ ]:
def quantile_code(x):
    
    if x < sum_amount_quantile[0.25]:
        return 1
    elif sum_amount_quantile[0.25] <= x < sum_amount_quantile[0.5]:
        return 2
    elif sum_amount_quantile[0.5] <= x < sum_amount_quantile[0.75]:
        return 3
    else:
        return 4   

df_055 = df_receipt[['customer_id', 'amount']].copy()
df_055_grouped = df_055.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()

sum_amount_quantile = df_055_grouped['sum_amount'].quantile([0.25, 0.5, 0.75, 1.0])
 
df_055_grouped['quantile_code'] = df_055_grouped['sum_amount'].apply(quantile_code)

df_055_grouped.head(10)

---
> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [ ]:
df_056 = df_customer[['customer_id', 'age', 'birth_day']].copy()
df_056['era'] = df_056['age'].apply(lambda x: '10代未満' if x < 10 else ('60代' if x > 60 else str(int(x / 10)) + '0代'))
df_056[['customer_id', 'birth_day', 'era']].head(10)

---
> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [ ]:
df_057 = df_customer[['customer_id', 'age', 'birth_day', 'gender']].copy()
df_057['era'] = df_057['age'].apply(lambda x: '10代未満' if x < 10 else ('60代' if x > 60 else str(int(x / 10)) + '0代'))
df_057['era + gender'] = df_057['era'] + df_057['gender']
df_057.head(10)

---
> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [ ]:
pd.get_dummies(df_customer[['customer_id', 'gender_cd']], columns=['gender_cd']).head(10)

---
> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

TIPS:
- query()の引数engineで'python'か'numexpr'かを選択でき、デフォルトはインストールされていればnumexprが、無ければpythonが使われます。さらに、文字列メソッドはengine='python'でないとquery()内で使えません。


In [ ]:
stdsc = preprocessing.StandardScaler()
df_059 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'][['customer_id', 'amount']].copy()
df_059_grouped = df_059.groupby('customer_id').agg(sum_amount = ('amount', 'sum'))
df_059_grouped['std_amount'] = stdsc.fit_transform(df_059_grouped[['sum_amount']].values)
df_059_grouped.head(10)

---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
mmsc = preprocessing.MinMaxScaler()
df_060 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'][['customer_id', 'amount']].copy()
df_060_grouped = df_060.groupby('customer_id').agg(sum_amount = ('amount', 'sum'))
df_060_grouped['mm_amount'] = mmsc.fit_transform(df_060_grouped[['sum_amount']].values)
df_060_grouped.head(10)

---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_061 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'][['customer_id', 'amount']].copy()
df_061_grouped = df_061.groupby('customer_id').agg(sum_amount = ('amount', 'sum'))
df_061_grouped['log10_amount'] = np.log10(df_061_grouped['sum_amount'])

---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_062 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'][['customer_id', 'amount']].copy()
df_062_grouped = df_062.groupby('customer_id').agg(sum_amount = ('amount', 'sum'))
df_062_grouped['log_amount'] = np.log(df_062_grouped['sum_amount'])

---
> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [ ]:
df_063 = df_product.copy()
df_063['profit'] = df_063['unit_price'] - df_063['unit_cost']
df_063.head(10)

---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [ ]:
df_064 = df_product.copy()

# 本問題では利益率 = (単価 - 原価) / 原価で算出
df_064['profit_rate'] = (df_064['unit_price'] - df_064['unit_cost']) / df_064['unit_price']
df_064['profit_rate'].mean()

---
> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
df_065 = df_product.copy()
df_065['pr30_unit_price'] = (df_065['unit_cost'] / 0.7).agg(np.floor)
df_065.head(10)

---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
df_066 = df_product.copy()
df_066['pr30_unit_price'] = (df_066['unit_cost'] / 0.7).round()
df_066.head(10)

---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
df_067 = df_product.copy()
df_067['pr30_unit_price'] = (df_067['unit_cost'] / 0.7).agg(np.ceil)
df_067.head(10)

---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [ ]:
df_068 = df_product.copy()
df_068['tax_included_unit_price'] = (df_068['unit_price'] * 1.1).agg(np.floor)
df_068.head(10)

---
> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [ ]:
df_069 = pd.merge(df_receipt, df_product, on = 'product_cd', how = 'inner')

df_069_grouped_tmp1 = df_069.groupby('customer_id').agg(sum_all_amount = ('amount', 'sum')).reset_index()

df_069_tmp2 = df_069[df_069['category_major_cd'] == '07'].copy()
df_069_grouped_tmp2 = df_069_tmp2.groupby('customer_id').agg(sum_07_amount = ('amount', 'sum')).reset_index()

df_069 = pd.merge(df_069_grouped_tmp1, df_069_grouped_tmp2, on = 'customer_id', how = 'right')
df_069['rate'] = df_069['sum_07_amount'] / df_069['sum_all_amount']
df_069.head(10)

---
> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [ ]:
df_070 = pd.merge(df_receipt, df_customer, on = 'customer_id', how = 'inner')
df_070['application_date'] = pd.to_datetime(df_070['application_date'])
df_070['sales_ymd'] = pd.to_datetime(df_070['sales_ymd'].astype(str))
df_070['date_diff'] = df_070['sales_ymd'] - df_070['application_date']
df_070[['customer_id', 'sales_ymd', 'application_date', 'date_diff']].head(10)

---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [ ]:
def diff_month(row):
    return monthmod(row['application_date'], row['sales_ymd'])[0].months
    
df_071 = pd.merge(df_receipt, df_customer, on = 'customer_id', how = 'inner')
df_071['application_date'] = pd.to_datetime(df_071['application_date'])
df_071['sales_ymd'] = pd.to_datetime(df_071['sales_ymd'].astype(str))
df_071['diff_month'] = df_071.apply(diff_month, axis = 1)
df_071[['customer_id', 'application_date', 'sales_ymd', 'diff_month']].head(10)

---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [ ]:
def diff_year(row):
    return monthmod(row['application_date'], row['sales_ymd'])[0].months // 12
    
df_071 = pd.merge(df_receipt, df_customer, on = 'customer_id', how = 'inner')
df_071['application_date'] = pd.to_datetime(df_071['application_date'])
df_071['sales_ymd'] = pd.to_datetime(df_071['sales_ymd'].astype(str))
df_071['diff_year'] = df_071.apply(diff_year, axis = 1)
df_071[['customer_id', 'application_date', 'sales_ymd', 'diff_year']].head(10)

---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [ ]:
df_073 = pd.merge(df_receipt, df_customer, on = 'customer_id', how = 'inner')
df_073['application_date'] = pd.to_datetime(df_073['application_date']).astype(int)
df_073['sales_ymd'] = pd.to_datetime(df_073['sales_ymd'].astype(str)).astype(int)
df_073['diff_epoch'] = (df_073['sales_ymd'] - df_073['application_date']) / 10 ** 9
df_073[['customer_id', 'application_date', 'sales_ymd', 'diff_epoch']].head(10)

---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [ ]:
def last_monday_ymd(row):
    return row['sales_ymd'] - timedelta(days = row['elapsed_day'])

df_074 = pd.merge(df_receipt, df_customer, on = 'customer_id', how = 'inner')
df_074['sales_ymd'] = pd.to_datetime(df_074['sales_ymd'].astype(str))
df_074['elapsed_day'] = df_074['sales_ymd'].apply(lambda x: x.weekday())
df_074['last_monday_ymd'] = df_074.apply(last_monday_ymd, axis = 1)
df_074[['sales_ymd', 'elapsed_day', 'last_monday_ymd']].head(10)

---
> P-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [ ]:
df_customer.sample(frac = 0.01).head(10)

---
> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [ ]:
train, test = train_test_split(df_customer, test_size=0.1, stratify = df_customer['gender_cd'])

test.groupby('gender_cd').agg(count_customer_id = ('customer_id', 'count')).reset_index()

---
> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [ ]:
df_077 = df_receipt[['customer_id', 'amount']].copy()
df_077_grouped = df_077.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()
df_077_grouped['log_amount'] = np.log(df_077_grouped['sum_amount'])

mean_log_amount = df_077_grouped['log_amount'].mean()
std_log_amount = df_077_grouped['log_amount'].std()

lower = mean_log_amount - 3 * std_log_amount
upper = mean_log_amount + 3 * std_log_amount

df_077_grouped[(df_077_grouped['log_amount'] < lower) | (df_077_grouped['log_amount'] > upper)].head(10)

---
> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [ ]:
df_078 = df_receipt[df_receipt['customer_id'].str[:1] != 'Z'].copy()
df_078_grouped = df_078.groupby('customer_id').agg(sum_amount = ('amount', 'sum')).reset_index()

q1 = df_078_grouped['sum_amount'].quantile(0.25)
q3 = df_078_grouped['sum_amount'].quantile(0.75)
iqr = q3 - q1

lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

df_078_grouped[(df_078_grouped['sum_amount'] < lower) | (df_078_grouped['sum_amount'] > upper)].head(10)

---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [ ]:
df_product.isnull().sum()

---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [ ]:
df_080 = df_product.dropna(how = 'any').copy()

print('削除前：' + str(len(df_product)))
print('削除後：' + str(len(df_080)))

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_081 = df_product.fillna(df_product.mean().round()).copy()
df_081.isnull().sum()

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_082 = df_product.fillna(df_product.median().round()).copy()
df_082.isnull().sum()

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_083 = df_product.copy()
df_083_grouped = df_product.groupby('category_small_cd').agg(median_unit_price = ('unit_price', 'median'), median_unit_cost = ('unit_cost', 'median'))

for category_small_cd, median_unit_price in df_083_grouped['median_unit_price'].items():
    df_083.loc[(df_083['category_small_cd'] == category_small_cd) & (df_083['unit_price'].isnull()), 'unit_price'] = median_unit_price
for category_small_cd, median_unit_cost in df_083_grouped['median_unit_cost'].items():
    df_083.loc[(df_083['category_small_cd'] == category_small_cd) & (df_083['unit_cost'].isnull()), 'unit_cost'] = median_unit_cost

df_083.isnull().sum()

---
> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [ ]:
df_084_all_r = df_receipt[['customer_id', 'amount']].copy()
df_084_all_r_grouped = df_084_all_r.groupby('customer_id').agg(all_amount = ('amount', 'sum')).reset_index()

df_084_2019_r = df_receipt[df_receipt['sales_ymd'].astype(str).str[:4] == '2019'][['customer_id', 'amount']].copy()
df_084_2019_r_grouped = df_084_2019_r.groupby('customer_id').agg(y2019_amount = ('amount', 'sum')).reset_index()
   
df_084 = pd.merge(pd.merge(df_customer['customer_id'], df_084_all_r_grouped, on = 'customer_id', how = 'left'),
                  df_084_2019_r_grouped, on = 'customer_id', how = 'left')

df_084['rate'] = df_084['y2019_amount'] / df_084['all_amount']
df_084.fillna(0, inplace = True)

df_084.head(10)

---
> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [ ]:
df_085_g = df_geocode[['postal_cd', 'longitude', 'latitude']].copy()
df_085_g_grouped = df_085_g.groupby('postal_cd').agg(mean_longitude = ('longitude', 'mean'), mean_latitude = ('latitude', 'mean')).reset_index()
df_085 = pd.merge(df_customer, df_085_g_grouped, on = 'postal_cd', how = 'inner')
df_085.head(10)

---
> P-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

In [ ]:
def calc_dist(x1, x2, y1, y2):
    dist = 6371 * math.acos(math.sin(math.radians(x1)) 
                * math.sin(math.radians(x2)) 
                + math.cos(math.radians(x1)) 
                * math.cos(math.radians(x2)) 
                * math.cos(math.radians(y1) - math.radians(y2)))
    return dist

df_086_g = df_geocode[['postal_cd', 'longitude', 'latitude']].copy()
df_086_g_grouped = df_086_g.groupby('postal_cd').agg(mean_longitude = ('longitude', 'mean'), mean_latitude = ('latitude', 'mean')).reset_index()
df_086 = pd.merge(df_customer[['customer_id', 'postal_cd', 'address', 'application_store_cd']], df_086_g_grouped, on = 'postal_cd', how = 'inner')
df_086.rename(columns = {'postal_cd': 'home_postal_cd', 'address': 'home_address',
                         'application_store_cd': 'store_cd', 'mean_longitude': 'home_longitude', 'mean_latitude': 'home_latitude'},
              inplace = True)

df_086_s = df_store[['store_cd', 'address', 'longitude', 'latitude']].copy()
df_086_s.rename(columns = {'address': 'store_address', 'longitude': 'store_longitude', 'latitude': 'store_latitude'}, inplace = True)
df_086 = pd.merge(df_086, df_086_s, on = 'store_cd', how = 'inner')
df_086['dist'] = df_086.apply(lambda x: calc_dist(x['home_latitude'], x['store_latitude'], x['home_longitude'], x['store_longitude']), axis = 1)

df_086[['customer_id', 'home_address', 'store_address', 'dist']].head(10)

---
> P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [ ]:
df_087_c_duplication = pd.merge(df_customer,
                                df_customer[df_customer.duplicated(subset = ['customer_name', 'postal_cd'])][['customer_name', 'postal_cd']],
                                on = ['customer_name', 'postal_cd'], how = 'inner')
df_087_r_duplication = pd.merge(df_receipt, df_087_c_duplication[['customer_id', 'customer_name', 'postal_cd']], on = 'customer_id', how = 'right')
df_087_r_duplication_grouped = df_087_r_duplication.groupby(['customer_id', 'customer_name', 'postal_cd']).agg(sum_amount = ('amount', 'sum')).reset_index()
df_087_r_duplication_grouped.sort_values(['customer_name', 'postal_cd', 'sum_amount', 'customer_id'], ascending=[False , False, False, True], inplace = True)
df_087_c_identified = df_087_r_duplication_grouped.drop_duplicates(subset = ['customer_name', 'postal_cd'], keep = 'first').copy()
df_087_c_identified['identified_flag'] = True

df_087 = pd.merge(df_customer, df_087_c_identified[['customer_id', 'identified_flag']], on = 'customer_id', how = 'left')
df_087.sort_values(['customer_name', 'postal_cd', 'identified_flag'], inplace = True)
df_087.drop_duplicates(subset = ['customer_name', 'postal_cd'], keep = 'first', inplace = True)

print('顧客データ数：' + str(len(df_customer)))
print('名寄顧客データ数：' + str(len(df_087)))
print('重複数データ数：' + str(len(df_087_c_identified)))

---
> P-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [ ]:
df_088_c_duplication = pd.merge(df_customer,
                                df_customer[df_customer.duplicated(subset = ['customer_name', 'postal_cd'])][['customer_name', 'postal_cd']],
                                on = ['customer_name', 'postal_cd'], how = 'inner')
df_088_r_duplication = pd.merge(df_receipt, df_088_c_duplication[['customer_id', 'customer_name', 'postal_cd']], on = 'customer_id', how = 'right')
df_088_r_duplication_grouped = df_088_r_duplication.groupby(['customer_id', 'customer_name', 'postal_cd']).agg(sum_amount = ('amount', 'sum')).reset_index()
df_088_r_duplication_grouped.sort_values(['customer_name', 'postal_cd', 'sum_amount', 'customer_id'], ascending=[False , False, False, True], inplace = True)
df_088_c_identified = df_088_r_duplication_grouped.drop_duplicates(subset = ['customer_name', 'postal_cd'], keep = 'first').copy()
df_088_c_identified['identified_flag'] = True
df_088_c_identified['identified_customer_id'] = df_088_c_identified['customer_id']

df_088 = pd.merge(df_customer, df_088_c_identified[['customer_id', 'identified_flag', 'identified_customer_id']], on = 'customer_id', how = 'left')
df_088.sort_values(['customer_name', 'postal_cd', 'identified_flag'], inplace = True)

df_088['identified_customer_id'].fillna(df_088['customer_id'], inplace = True)

print('顧客IDのユニーク件数：' + str(df_088['customer_id'].nunique()))
print('統合名寄IDのユニーク件数：' + str(df_088['identified_customer_id'].nunique()))

---
> P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [ ]:
df_089_r = df_receipt['customer_id'].drop_duplicates().copy()
df_089 = pd.merge(df_customer, df_089_r, on = 'customer_id', how = 'inner')

df_train, df_test = train_test_split(df_089, test_size=0.2)

---
> P-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

In [ ]:
df_090 = df_receipt[['sales_ymd', 'amount']].copy()
df_090['sales_ym'] = df_receipt['sales_ymd'].astype(str).str[:6]
df_090_grouped = df_090.groupby('sales_ym').agg(sum_amount = ('amount', 'sum')).reset_index()

df_train = []
df_test = []

tss = TimeSeriesSplit(n_splits = 3, max_train_size = 12, test_size = 6)
for train, test in tss.split(df_090_grouped):
    df_train.append(df_090_grouped.loc[train, :])
    df_test.append(df_090_grouped.loc[test, :])

---
> P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [ ]:
df_091_r = df_receipt.drop_duplicates(subset = 'customer_id').copy()
df_091_r['buy_flag'] = True

df_091 = pd.merge(df_customer, df_091_r[['customer_id', 'buy_flag']], on = 'customer_id', how = 'left')

df_091_buy = df_091[df_091['buy_flag'] == True]
df_091_not_buy = df_091[df_091['buy_flag'] != True].sample(len(df_091_buy))

print('売上実績有顧客：' + str(len(df_091_buy)))
print('売上実績無顧客：' + str(len(df_091_not_buy)))

---
> P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

In [ ]:
df_092_c = df_customer.drop(columns = 'gender')
df_092_g = df_customer[['gender_cd', 'gender']].drop_duplicates()

---
> P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [ ]:
df_093_c = df_category[['category_major_name', 'category_medium_name', 'category_small_name', 'category_small_cd']]
df_093 = pd.merge(df_product, df_093_c, on = 'category_small_cd', how = 'inner')

---
> P-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [ ]:
df_093.to_csv('data/df_094.csv', encoding = 'utf-8', index = False)

---
> P-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [ ]:
df_093.to_csv('data/df_095.csv', encoding = 'cp932', index = False)

---
> P-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [ ]:
df_093.to_csv('data/df_096.csv', encoding = 'utf-8', index = False, header = False)

---
> P-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

In [ ]:
df_097 = pd.read_csv('data/df_094.csv', encoding = 'utf-8')
df_097.head(3)

---
> P-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

In [ ]:
df_098 = pd.read_csv('data/df_096.csv', encoding = 'utf-8', header = None)
df_098.head(3)

---
> P-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [ ]:
df_093.to_csv('data/df_099.tsv', encoding = 'utf-8', sep = '\t', index = False)

---
> P-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

In [ ]:
df_100 = pd.read_csv('data/df_099.tsv', encoding = 'utf-8', sep = '\t')
df_100.head(3)

# これで１００本終わりです。おつかれさまでした！